In [10]:
# This is the generic code for health impact assessment (HIA) and burden of disease (BoD) calculations
# Forked originally from http://en.opasnet.org/w/HIA
# Original R code described in more detail at https://github.com/jtuomist/ghg-notebooks/wiki/Health-impact-assessment

import pandas as pd
import numpy as np

gas = ['CO2','CH4']

In [176]:
def concentration():
    df = pd.DataFrame({
        'exposure_agent':gas,
        'result':[1,2] # mg/kg, ppm, or similar
    })
    return df.set_index('exposure_agent')

def consumption():
    df = pd.DataFrame({
        'exposure_agent':gas,
        'result':[2,3]
    })
    return df.set_index('exposure_agent')

def exposure(
    p1 = concentration(),
    p2 = consumption()):
    
    out = p1 * p2
    #colnames(out@output)[colnames(out@output) == "Pathogen"] <- "Exposure_agent"

    return out

def bw():
    df = pd.DataFrame({
        'sex':['male','female'],
        'result':[75,65]
    })
    return df.set_index('sex')

def dose(
    p1 = exposure(),
    p2 = bw()):
    
    outlog = np.log10(p1).assign(scaling='log10').reset_index()

    outbw = p1.assign(tmp=1).set_index('tmp', append=True)
    outbw = outbw * p2.assign(tmp=1).set_index('tmp', append=True)
    outbw = outbw.assign(scaling='bw').reset_index().drop('tmp', axis=1)

    out = p1.reset_index().assign(scaling='none').append(outlog).append(outbw)

    return out.set_index(['exposure_agent','scaling','sex'])


display(dose())

result
exposure_agent scaling sex               
CO2            none    NaN       2.000000
CH4            none    NaN       6.000000
CO2            log10   NaN       0.301030
CH4            log10   NaN       0.778151
CO2            bw      male    150.000000
                       female  130.000000
CH4            bw      male    450.000000
                       female  390.000000

In [ ]:
#http://en.opasnet.org/w/Special:Opasnet_Base?id=op_en5827
        
def erf():
    df = pd.DataFrame({
        'erf':[1,2],
        'result':[2,3]
    })

    return df.set_index('erf')

"""
    RR
    function (...)
    {
    t1 <- t2 <- Ovariable()
    clean <- function(ova, obs) {
    parameter <- c("ERF", "Threshold")
    ova <- ova[ova$Observation == parameter[obs], colnames(ova@output) !=
    "Observation"]
    colnames(ova@output)[colnames(ova@output) == paste0(ova@name,
    "Result")] <- "Result"
    ova@name <- character()
    return(ova)
    }
    tmp <- ERF[ERF$ER_function %in% c("RR", "OR"), ]
    if (nrow(tmp@output) > 0) {
    rr <- clean(tmp, 1)
    threshold <- clean(tmp, 2)
    if ("OR" %in% tmp$ER_function)
    warning("OR is simply assumed to equal RR, which is fair approximation with small OR and low incidence.\n")
    dose2 <- dose - threshold
    result(dose2) <- pmax(0, result(dose2))
    t1 <- tryCatch(exp(log(rr) * dose2), error = function(e) return(Ovariable()))
    }
    tmp <- ERF[ERF$ER_function %in% c("Relative Hill"), ]
    if (nrow(tmp@output) > 0) {
    Imax <- clean(tmp, 1)
    ed50 <- clean(tmp, 2)
    t2 <- tryCatch(1 + (dose * Imax)/(dose + ed50), error = function(e) return(Ovariable()))
    }
    out <- orbind(t1@output, t2@output)
    if (nrow(out) == 0) {
    out <- data.frame(Result = 1)
    }
    return(out)
    }

    Frexposed is the fraction of exposed individuals within the target population. Defaults to 1 but may be indexed by population subgroups.

    frexposed
    function (...)
    {
    0
    }

    P_illness is the probability of illness. Relevant for microbial infection endpoints. Typically a microbe-specific constant.

    P_illness
    function (...)
    {
    0
    }

    mc2d is a function that samples the ovariable that describes individuals and then aggregates (typically averages over) the samples to reflect the situation of a defined population or population subgroups. This is done because in disease burden assessments, we are typically interested in population-level uncertainties rather than individual uncertainties.

    mc2d
    function (ova, mc2dpar = NULL)
    {
    if (is.null(mc2dpar))
    if (exists("mc2dparam"))
    mc2dpar <- mc2dparam
    else stop("Parameter list mc2dparam missing!\n")
    if (mc2dpar$run2d) {
    ova <- ova * mc2dpar$info
    require(reshape2)
    marg <- setdiff(c(colnames(ova@output)[ova@marginal],
    mc2dpar$newmarginals), "Iter")
    out <- aggregate(result(ova), by = ova@output[colnames(ova@output) %in%
    marg], FUN = function(x) {
    strength <- if (is.null(mc2dpar$strength))
    length(x)
    else mc2dpar$strength
    apply(array(as.numeric(sample(as.character(x), strength *
    mc2dpar$N2, replace = TRUE)), dim = c(strength,
    mc2dpar$N)), MARGIN = 2, FUN = mc2dpar$fun)
    })
    temp <- melt(out[[length(out)]])
    out[[length(out)]] <- 1:nrow(out)
    colnames(temp) <- c("Nrow", "Iter", "Result")
    out <- merge(out, temp, by.x = "x", by.y = "Nrow")
    out$x <- NULL
    out <- Ovariable(output = out, marginal = colnames(out) %in%
    c(marg, "Iter"))
    }
    else {
    out <- ova
    }
    return(out)
    }

    sumExposcen subtracts the PAF of BAU scenario from the scenario of interest, thus giving the impact the exposure. The actual subtracting is done after the ovariable is evaluated by using CollapseMarginal (a standard function to manipulate ovariables in OpasnetUtils).

    sumExposcen
    function (out)
    {
    if ("Exposcen" %in% colnames(out@output)) {
    out <- out * Ovariable(output = data.frame(Exposcen = c("BAU",
    "No exposure"), Result = c(1, -1)), marginal = c(TRUE,
    FALSE))
    }
    return(out)
    }
"""
